In [ ]:
from pathlib import Path
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rasterio
from rasterio.session import AWSSession

In [ ]:
# Pfade definieren
base_dir = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd() / "data"
co2 = base_dir / "co2_mm_mlo.csv"
sea_level = base_dir / "epa_sea_level.csv"
tempanomaly = base_dir / "gistemp1200_GHCNv4_ERSSTv5.nc"

# Existenz aller Dateien überprüfen
files_to_check = [co2, sea_level, tempanomaly]
missing = [str(f) for f in files_to_check if not f.exists()]
if missing:
    raise FileNotFoundError(f"Folgende Dateien fehlen: {', '.join(missing)}")

# Daten einlesen
df_co2 = pd.read_csv(co2, comment="#", on_bad_lines="skip", engine="python")
df_sea_level = pd.read_csv(sea_level, comment="#", on_bad_lines="skip", engine="python")
dataset_tempanomaly = xr.open_dataset(tempanomaly)

# Die direkte S3-URL für die Kachel Düsseldorf/Rheinland
aws_url_topography = "s3://copernicus-dem-30m/Copernicus_DSM_COG_10_N51_00_E006_00_DEM/Copernicus_DSM_COG_10_N51_00_E006_00_DEM.tif"


In [ ]:
# Zeitspalte erstellen und als Index setzen
df_co2['date'] = pd.to_datetime(df_co2[['year','month']].assign(day=1))
df_co2 = df_co2.set_index('date').sort_index()

# Liniendiagramm erstellen
plt.figure(figsize=(12,5))
plt.plot(df_co2.index, df_co2['average'], color='green', alpha=0.6, label='Monatliche Messung')
plt.plot(df_co2.index, df_co2['deseasonalized'], color='red', alpha=0.6, label='Deseisonalisierte Messung')
plt.title('CO₂-Konzentration über die Zeit')
plt.xlabel('Datum')
plt.ylabel('CO₂ (ppm)')
plt.grid(alpha=0.2)
plt.legend()
plt.show()

In [ ]:
# Zeitspalte erstellen und als Index setzen
df_sea_level['date'] = pd.to_datetime(df_sea_level['Year'].astype(int).astype(str), format='%Y')
df_sea_level = df_sea_level.set_index('date').sort_index()

# Einheiten von Zoll (inch) in Meter umrechnen
inch_to_cm = 2.54
for col in ['Lower Error Bound', 'Upper Error Bound', 'CSIRO Adjusted Sea Level']:
    df_sea_level[col] = df_sea_level[col].astype(float) * inch_to_cm

# Liniendiagramm erstellen
plt.figure(figsize=(12,6))
plt.fill_between(df_sea_level.index, df_sea_level['Lower Error Bound'], df_sea_level['Upper Error Bound'], color='lightblue', alpha=0.4, label='Fehlerbereich')
plt.plot(df_sea_level.index, df_sea_level['CSIRO Adjusted Sea Level'], label='CSIRO angepasst', color='tab:blue', alpha=0.8)
plt.plot(df_sea_level['CSIRO Adjusted Sea Level'].rolling(10, center=True).mean(), color='tab:red', linewidth=2, label='5‑Jahres‑Mittel (CSIRO)')
plt.title('Meeresspiegel (angepasst) mit Fehlerbereichen')
plt.xlabel('Jahr')
plt.ylabel('Meeresspiegel (cm)')
plt.grid(alpha=0.2)
plt.legend()
plt.show()

In [ ]:
# Jahr für die Auswahl der Daten
year = 2024

# Temperaturanomalien für einen bestimmten Zeitpunkt auswählen
tempanomaly = dataset_tempanomaly["tempanomaly"].sel(time=f"{year}-01-15")

# Plot erstellen
fig, ax = plt.subplots(
    figsize=(12, 8),
    subplot_kw={"projection": ccrs.PlateCarree()}  # Kartenprojektion
)

# Weltkarte hinzufügen
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)  # Küstenlinien
ax.add_feature(cfeature.BORDERS, linewidth=0.5)    # Ländergrenzen
ax.add_feature(cfeature.LAND, facecolor="lightgray")  # Landflächen
ax.add_feature(cfeature.OCEAN, facecolor="lightblue")  # Ozeanflächen

# Temperaturanomalien plotten
tempanomaly.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),  # Projektion der Daten
    cmap="coolwarm",               # Farbskala
    cbar_kwargs={"label": "Temperaturanomalie (°C)"},  # Farblegende
)

# Titel hinzufügen
ax.set_title(f"Temperaturanomalien am 15. Januar {year}", fontsize=14)

# Plot anzeigen
plt.show()

In [ ]:
# Umgebung mit anonymem Zugriff starten, Daten verkleinert (512x512) laden und direkt plotten
with rasterio.Env(AWSSession(aws_unsigned=True)):
    with rasterio.open(aws_url_topography) as src:
        plt.imshow(src.read(1, out_shape=(512, 512)), cmap='terrain')
        plt.colorbar(label='Höhe über NN (m)')
        plt.title("Topografie Region Düsseldorf (N51, E006)")
        plt.show()